# `torch.backends` 
——它是 PyTorch 的底层计算库控制接口。

- 在 PyTorch 中，很多数值计算（特别是矩阵运算、卷积等）并不是由 `PyTorch` 自己实现的，而是调用了 `高性能的第三方库`，比如：

    - `cuDNN`（NVIDIA 提供的 GPU 深度学习加速库）
    
    - `MKL` / `OpenBLAS`（CPU 数学库）
    
    - `CUDA`（NVIDIA GPU 通用计算平台）

而` torch.backends `就是一个 `开关/配置入口`，让我们能够 控制这些底层库的行为，在 `性能` 和 `精度` 之间做平衡。

---
## 1. 模块结构

- `torch.backends` 下有几个子模块，常用的是：

- `torch.backends.cudnn` → 控制` cuDNN `库（GPU 加速）

- `torch.backends.mkl` → 控制` MKL `库（Intel CPU 加速）

- `torch.backends.openmp`→ 控制` OpenMP`（多线程并行计算）

- `torch.backends.cuda `→ 一些` CUDA `相关选项（较少用）

---
## 2. 重点：`torch.backends.cudnn`

这是`最常用的部分`，因为` cuDNN `对`卷积`、`RNN` 等 `GPU` 操作有重大加速作用。

常见属性:
| 属性                                   | 类型   | 作用                                      |
| ------------------------------------ | ---- | --------------------------------------- |
| `torch.backends.cudnn.enabled`       | bool | 是否启用 cuDNN（默认 True，有 GPU 时建议开启）         |
| `torch.backends.cudnn.benchmark`     | bool | 让 cuDNN 自动寻找最优卷积算法（适合输入尺寸固定的情况）         |
| `torch.backends.cudnn.deterministic` | bool | 强制使用确定性算法（结果可复现，但可能降低性能）                |
| `torch.backends.cudnn.allow_tf32`    | bool | 是否允许使用 TensorFloat-32（在支持的 GPU 上提高训练速度） |

---
使用示例

In [ ]:
import torch

# 启用 cuDNN
torch.backends.cudnn.enabled = True  

# 适合固定输入大小（如图像分类），提升性能
torch.backends.cudnn.benchmark = True  

# 为了保证结果可复现
torch.backends.cudnn.deterministic = True  


⚠️ 注意：

`benchmark`=True 会尝试不同的卷积算法并选择最快的，但输入 shape 改变时会重新搜索，可能拖慢速度。

`deterministic`=True 会关闭某些非确定性算法，保证实验可复现。

## 3. 其他子模块
### （1）torch.backends.mkl

针对 Intel MKL 库（CPU 数学库）。
常见：

In [ ]:
torch.backends.mkl.is_available()   # 是否支持 MKL
torch.backends.mkl.is_enabled()     # 是否启用 MKL


### （2）torch.backends.openmp

控制 OpenMP（多线程 CPU 并行）。

In [ ]:
torch.backends.openmp.is_available()  # 是否支持 OpenMP
torch.backends.openmp.is_enabled()    # 是否启用 OpenMP


### （3）torch.backends.cuda

一般很少直接用，但里面有一些低级配置。

---
## 4. 常见使用场景

1、加速训练（benchmark 模式）

In [ ]:
torch.backends.cudnn.benchmark = True


适合 输入大小固定（如 ImageNet 分类）。

---
2、结果可复现

In [ ]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


适合科研/实验，保证每次运行结果一致。

---
3、兼容性调试 \
如果遇到某些 cuDNN 算法导致 NaN 或不收敛，可以直接禁用：

In [ ]:
torch.backends.cudnn.enabled = False


## ✅ 总结：
torch.backends 是 PyTorch 提供的 底层性能/兼容性开关，常用于：

- cudnn.enabled → 是否用 cuDNN

- cudnn.benchmark → 动态选择最优算法（快但不稳定）

- cudnn.deterministic → 强制确定性计算（可复现但慢）

- mkl / openmp → CPU 底层库支持